# Spaceship Titanic Classification w/ Binary Logistic Regression

#### Dataset: https://www.kaggle.com/competitions/spaceship-titanic/overview
##### Dataset License: https://creativecommons.org/licenses/by/4.0/

###### Author: Cody Weaver

### Load and Process Dataset

In [1]:
import numpy as np
import pandas as pd
from lr import LogisticRegressionModel

In [2]:
# load raw dataset
raw_data = pd.read_csv('../data/train.csv')
print(raw_data.head())
print(raw_data.dtypes)

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  
Pa

#### Classification using amount billed for amenities only

##### Normalize data

In [3]:
amenities_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# filter to only include amenities billing data and passenger ID
amenities_data = raw_data[['PassengerId'] + amenities_columns + ['Transported']].copy()
print(amenities_data.head())

# fill in missing values for amenities
amenities_data[amenities_columns] = amenities_data[amenities_columns].fillna(0)

# normalize to mean zero and unit variance
for column in amenities_columns:
    column_mean = amenities_data[column].mean()
    column_std = amenities_data[column].std()
    normalize = lambda x: (x - column_mean) / column_std
    amenities_data[column] = normalize(amenities_data[column])

# covert boolean labels to 0-1 labels
amenities_data['Transported'] = amenities_data['Transported'].apply(
        lambda label: 0 if label == False else 1
    )

  PassengerId  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0     0001_01          0.0        0.0           0.0     0.0     0.0   
1     0002_01        109.0        9.0          25.0   549.0    44.0   
2     0003_01         43.0     3576.0           0.0  6715.0    49.0   
3     0003_02          0.0     1283.0         371.0  3329.0   193.0   
4     0004_01        303.0       70.0         151.0   565.0     2.0   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


##### Train Model

In [4]:
amenities_model = LogisticRegressionModel(len(amenities_columns))
num_epochs = 10
amenities_model.fit_model(
    amenities_data[amenities_columns].to_numpy(),
    amenities_data['Transported'],
    num_epochs=num_epochs,
)

Epoch: 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8693/8693 [00:00<00:00, 119205.29it/s]


Average Log Loss for epoch 0: 0.69


Epoch: 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8693/8693 [00:00<00:00, 111532.14it/s]


Average Log Loss for epoch 1: 0.68


Epoch: 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8693/8693 [00:00<00:00, 110531.01it/s]


...

Average Log Loss for epoch 7: 0.64


Epoch: 8: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8693/8693 [00:00<00:00, 100988.49it/s]


Average Log Loss for epoch 8: 0.64


Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8693/8693 [00:00<00:00, 116073.74it/s]

Average Log Loss for epoch 9: 0.63


In [7]:
X = amenities_data[amenities_columns].to_numpy()
Y = amenities_data['Transported']
print(amenities_model.evaluate(X, Y))

0.7013689175198435
